# Select sgRNA for Library


## Add 'InLibrary'

To indicate which sgRNAs are included in the library, a new column 'InLibrary' was added to the SingleGuideRNA table. 

In [ ]:
import data_processing as dp

def add_inlibrary(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    
    db_con.add_column("InLibrary", "VARCHAR(1)", "SingleGuideRNA")
    
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
add_inlibrary("miR-test", firewall=True)

## Find Exactly 4 or 5

First, those miRNAs with exactly 4 or 5 sgRNAs which are not excluded were identified and included in the library.

In [ ]:
import data_processing as dp

def find_exact(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    
    rows = db_con.fetch_query("""SELECT p.PriID
FROM SgRNATargetInformation AS t
JOIN PrimaryMicroRNA AS p
ON t.PriID = p.PriID
JOIN (SELECT SgID FROM SingleGuideRNA WHERE Exclude IS NULL) AS s
ON t.SgID = s.SgID
GROUP BY p.PriID
HAVING COUNT(DISTINCT s.SgID) = 5 OR COUNT(DISTINCT s.SgID) = 4""")
    
    if sql_version == "MSSQL":
        pris = ["'{}'".format(row.PriID) for row in rows]
    else:
        pris = ["'{}'".format(row) for row, in rows]
    pri_str = ",".join(pris)
        
    # fetch sgRNAs associated with these miRNAs
    sg_rows = db_con.fetch_query("""SELECT s.SgID
FROM SingleGuideRNA AS s
JOIN SgRNATargetInformation AS t
ON s.SgID = t.SgID
WHERE s.Exclude IS NULL AND t.PriID IN ({});""".format(pri_str))
    
    if sql_version == "MSSQL":
        sgs = [row.SgID for row in sg_rows]
    else:
        sgs = [sg for sg, in sg_rows]
    
    # add 'T' to InLibrary
    db_con.update_many_rows({"InLibrary": ['T']*len(sgs)}, {"SgID": sgs}, "SingleGuideRNA")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
find_exact("miR-test", firewall=True)

This adds 1059 sgRNAs to the library which target 268 primary miRNAs (257 with at least 4 sgRNAs).


## Find the Rest

Those primary miRNAs with more than 5 good sgRNAs were then filtered to identify the best possible sgRNAs.

In [ ]:
import data_processing as dp

def find_all(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
        
    rows = db_con.fetch_query("""SELECT p.PriID
FROM SgRNATargetInformation AS t
JOIN PrimaryMicroRNA AS p
ON t.PriID = p.PriID
JOIN (SELECT SgID FROM SingleGuideRNA WHERE Exclude IS NULL) AS s
ON t.SgID = s.SgID
GROUP BY p.PriID
HAVING COUNT(DISTINCT s.SgID) > 5""")
    
    if sql_version == "MSSQL":
        pris = ["'{}'".format(row.PriID) for row in rows]
    else:
        pris = ["'{}'".format(row) for row, in rows]
    pri_str = ",".join(pris)
        
    # fetch sgRNAs associated with these miRNAs
    sg_rows = db_con.fetch_query("""SELECT t.PriID, s.SgID, s.MaxAzimuthScore
FROM SingleGuideRNA AS s
JOIN SgRNATargetInformation AS t
ON s.SgID = t.SgID
WHERE s.Exclude IS NULL AND t.PriID IN ({}) AND s.SgID NOT IN (
SELECT OverlappingSgID FROM OverlappingSgRNAs);""".format(pri_str))
    pri_dict = {}
    for row in sg_rows:
        if sql_version == "MSSQL":
            pri = row.PriID
            sg = row.SgID
            score = row.MaxAzimuthScore
        else:
            pri, sg, score = row
            pri = str(pri)
        if pri not in pri_dict:
            pri_dict[pri] = [(score, sg)]
        else:
            pri_dict[pri] += [(score, sg)]
    
    sg_list = []
    for key, val in pri_dict.iteritems():
        # sort by score
        val = sorted(val, reverse=True)
        # make sure have five sgRNAs per miRNA
        if len(val) < 5:
            print "Something went wrong with PriID {}".format(key)
        else:
            pri_sg_list = [sg for score, sg in val[:5]]
            sg_list += pri_sg_list
    
    db_con.update_many_rows({"InLibrary": ['T']*len(sg_list)}, {"SgID": sg_list}, "SingleGuideRNA")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
find_all("miR-test", firewall=True)

The number of sgRNAs per primary miRNA can then be fetched.

In [ ]:
db_con = dp.DatabaseConnection("MySQL", db_name="miR-test", firewall=True)
df = db_con.fetch_query_as_df("""SELECT t.PriID, COUNT(s.SgID) AS NumSg
FROM (SELECT SgID FROM SingleGuideRNA WHERE InLibrary LIKE 'T') AS s 
JOIN SgRNATargetInformation AS t 
ON s.SgID = t.SgID
GROUP BY t.PriID
ORDER BY COUNT(s.SgID)""", "PriID")
db_con.close_cursor()
db_con.close_connection()

df.shape

In [ ]:
df[df["NumSg"] >= 4].shape

This results in 7382 sgRNAs targeting 1599 primary miRNAs (1594 with 4 or more sgRNAs).